In [1]:
import cv2
import os
import re
import matplotlib.pyplot as plt
import math
import time
import mediapipe as mp
import numpy as np
import Utils as utils

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# path = 'C://Users//juan.guevara01//Documents//Tesis//Videos//VideosInstitutoCreSer//Lsec_001'
# path_test = 'Videos//FrameTrain'

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
#     mp_drawing.draw_landmarks(image, 
#                             results.pose_landmarks, 
#                             mp_holistic.POSE_CONNECTIONS) # Draw pose connections

    mp_drawing.draw_landmarks(image, 
                              results.right_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, 
                              results.left_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(234,23,10), thickness=1, circle_radius=1))
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks.landmark else np.zeros(132)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])

def extract_keypoints_V3(results):
    rh_x = [res.x for res in results.right_hand_landmarks.landmark] if results.right_hand_landmarks else []
    lh_x = [res.x for res in results.left_hand_landmarks.landmark] if results.left_hand_landmarks else []
    rh_y = [res.y for res in results.right_hand_landmarks.landmark] if results.right_hand_landmarks else []
    lh_y = [res.y for res in results.left_hand_landmarks.landmark] if results.left_hand_landmarks else []
    if(lh_y and lh_x):
        if(min(lh_y) >= 1):
            lh_y = []
        if(min(lh_x) >= 1):
            lh_x = []
    if(rh_y and rh_x):
        if(min(rh_y) >= 1):
            rh_y = []
        if(min(rh_x) >= 1):
            rh_x = []
    return rh_x,rh_y,lh_x,lh_y

def find_max_min(keypoints_x, keypoints_y):
    x_min = int(min(keypoints_x)*480-5)
    x_max = int(max(keypoints_x)*480+5)
    y_min = int(min(keypoints_y)*360-5)
    y_max = int(max(keypoints_y)*360)
    if(x_min < 0):
        x_min = 0
    if(y_min < 0):
        y_min = 0
    if(y_max > 360):
        y_max = 360
    if(x_max > 480):
        x_max = 480
    return x_min, x_max, y_min, y_max

def save_img(keypoints_x, keypoints_y, image, count_1, count_2):
    x_min, x_max, y_min, y_max = find_max_min(keypoints_x,keypoints_y)
    hand_image = cv2.resize(image[y_min:y_max , x_min:x_max], (50,50))
    filename = "TestData/Lsec_"+str(count_1)+"/"+str(count_2)+".jpg"
    cv2.imwrite(filename, hand_image)

In [60]:
pathOrigen = 'C:\\Users\\juan.guevara01\\Documents\\Tesis\\Videos\\Version1\\Validating\\'
directory_list = os.listdir(pathOrigen)
count_1 = 6
count_2 = 0

In [61]:
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as holistic:
    for directory in directory_list:
        videoFiles = sorted_alphanumeric(os.listdir(pathOrigen+directory))
        count_2 = 0
        for videoFile in videoFiles:
            cap = cv2.VideoCapture(pathOrigen+'\\'+directory+'\\'+videoFile)
            while(cap.isOpened()):
                frameId = cap.get(1)
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % 2 == 0):
                    if(not frame.shape == (360, 480, 3)):
                        frame = cv2.resize(frame, (480,360))
                    image, results = mediapipe_detection(frame, holistic)
                    rh_keypoints_x, rh_keypoints_y, lh_keypoints_x, lh_keypoints_y = extract_keypoints_V3(results)
                    if(rh_keypoints_x and rh_keypoints_y):
                        save_img(rh_keypoints_x, rh_keypoints_y, image, count_1, count_2)
                        count_2 += 1
                    if(lh_keypoints_x and lh_keypoints_y):
                        save_img(lh_keypoints_x, lh_keypoints_y, image, count_1, count_2)
                        count_2 += 1
            cap.release()
        count_1 += 1
print("Done!")

Done!


# Modo Reducido

In [4]:
pathOrigen = 'C:\\Users\\juan.guevara01\\Documents\\Tesis\\Videos\\Version1\\Validating\\'
pathDestino = 'DataSet/Validating'
videoFiles = utils.sorted_alphanumeric(os.listdir(pathOrigen))
count_1 = 0
count_2 = 0

In [6]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [7]:
def save_img(keypoints_x, keypoints_y, image, filename):
    x_min, x_max, y_min, y_max = utils.find_max_min(keypoints_x,keypoints_y)
    hand_image = cv2.resize(image[y_min:y_max , x_min:x_max], (50,50))
    cv2.imwrite(filename, hand_image)

In [8]:
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as holistic:
    for directory in utils.sorted_alphanumeric(os.listdir(pathOrigen)):
        count_1 += 1
        for video_file in utils.sorted_alphanumeric(os.listdir(pathOrigen+directory)):
            cap = cv2.VideoCapture(pathOrigen+directory+'\\'+video_file)
            while(cap.isOpened()):
                frameId = cap.get(1)
                ret, frame = cap.read()
                if (ret != True):
                    break
                if(not frame.shape == (360, 480, 3)):
                    frame = cv2.resize(frame, (480,360))
                image, results = utils.mediapipe_detection(frame, holistic)
                rh_keypoints_x, rh_keypoints_y, lh_keypoints_x, lh_keypoints_y = utils.extract_keypoints_V3(results)
                if(rh_keypoints_x and rh_keypoints_y):
                    path_destino = pathDestino+'/Lsec_'+str(count_1)+'/'+str(count_2)+'.jpg'
                    save_img(rh_keypoints_x, rh_keypoints_y, image, path_destino)
                    count_2 += 1
                if(lh_keypoints_x and lh_keypoints_y):
                    path_destino = pathDestino+'/Lsec_'+str(count_1)+'/'+str(count_2)+'.jpg'
                    save_img(lh_keypoints_x, lh_keypoints_y, image, path_destino)
                    count_2 += 1
            cap.release()
print("Done!")

Done!
